In [76]:
from tkinter import *
import colorama as clma
import ipywidgets as wg
from IPython.display import display
from tkinter.ttk import Combobox
from tkinter import messagebox
from tkPDFViewer import tkPDFViewer as pdf
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import plotly.graph_objects as go
import pandas as pd
import pandas_profiling as pp


In [71]:
file = pd.read_csv(r"C:\Users\Blazej\anaconda3\Python\jdszr6-materialy-python-master\stroke\healthcare-dataset-stroke-data.csv")

mediana = file["bmi"].median()
file.fillna(mediana, inplace=True) #Po sprawdzeniu NaN tylko w "bmi". fillna wstawia mediane, inplace=true 'nadpisuje' na aktualny dataframe 

# Zamieniam hypertension i heart_disease na wartości Yes/No
file["hypertension"] = file["hypertension"].replace(0, "No").replace(1, "Yes")
file["heart_disease"] = file["heart_disease"].replace(0, "No").replace(1, "Yes")

# #dodaje kolumnę z przedziałami wiekowymi
file.loc[(file["age"] < 20.0), "age_group"] = "nastolatki"
file.loc[(file["age"] >= 20) & (file["age"] <30), "age_group"] = "20latki"
file.loc[(file["age"] >= 30) & (file["age"] <40), "age_group"] = "30latki"
file.loc[(file["age"] >= 40) & (file["age"] <50), "age_group"] = "40latki"
file.loc[(file["age"] >= 50) & (file["age"] <60), "age_group"] = "50latki"
file.loc[(file["age"] >= 60) & (file["age"] <70), "age_group"] = "60latki"
file.loc[file["age"] >= 70, "age_group"] = "70+"

In [ ]:
file

In [81]:
# Aplikacja

window = Tk()
window.title("World Data Analysis")
window.geometry("500x440")
window.resizable(False, False)
window.config(bg = "light blue")
window.iconbitmap("C:/Users/Blazej/anaconda3/Python/jdszr6-materialy-python-master/stroke/WDA.ico")
heading = Label(text="HEALTH APPLICATION", bg="navy", fg="light blue", width="500", height="3", font="Helvetica 20 bold")
heading.pack()

Text_frame_1 = LabelFrame(window, text="Application description", font = "Helvetica 10 bold", bg = "light blue", borderwidth = 3)
Text_frame_1.place(x=25, y=110)
interview_description = Label(Text_frame_1, bg = "light blue" ,text="""
Welcome in health application. Our program allows you to calculate probability of 
having a stroke. You will need to answer on a few questions.

Please click "Open form" button when you are ready to start.
""").pack()

gender_options = [
    "Male",
    "Female"
]

yes_no = [
    "Yes",
    "No"
]

work_type_options = [
    "Private",
    "Self-employed",
    "Govt_job",
    "Children",
    "Never_worked"
]

residence_type_options = [
    "Urban",
    "Rural"
]

smoking_status_options = [
    "Former smoker",
    "Never smoked",
    "Smokes",
    "Unknown"
]

gender = StringVar()
gender.set(1)
age = IntVar()
hypertension = StringVar()
heart_disease = StringVar()
ever_married = StringVar()
residence_type = StringVar()
work_type = StringVar()
avg_glucose_level = IntVar()
bmi = IntVar()
height = IntVar()
weight = IntVar()
height_entry = IntVar()
weight_entry = IntVar()
smoking_status = StringVar()
#stroke = StringVar()

results = {"gender": gender,
            "age": age,
            "hypertension": hypertension,
            "heart disease": heart_disease,
            "ever married": ever_married,
            "work type": work_type,
            "residence type": residence_type,
            "avg glucose level": avg_glucose_level,
            "bmi": bmi,
            "smoking status": smoking_status
            #"stroke": stroke
            }

def check_age():
    if age_info >= 0:
        return age_info
    else:
        age_warning = messagebox.showinfo("WARNING", "You can't insert negative age!")
        return age_warning

def get_data():
    global age_info

    gendername_info = gender.get()

    try:
        age_info = age.get()
    except TclError:
        messagebox.showinfo("WARNING", "Please, insert numbers only in AGE field!")

    check_age()
    
    hypertension_info = hypertension_entry.get()
    heart_disease_info = heart_disease_entry.get()
    ever_married_info = ever_married_entry.get()
    work_type_info = work_type_entry.get()
    residence_type_info = residence_type_entry.get()

    try: 
        avg_glucose_level_info = avg_glucose_level.get()
    except TclError:
        messagebox.showinfo("WARNING", "Please insert numebrs only in AVERAGE GLUCOSE LEVEL field!")
        
    #wprowadzam samo bmi_info = bmi.get()   try except wyłączone 
    #bmi_info = bmi.get()
    try:
        bmi_info = bmi.get()
    except TclError:
        messagebox.showinfo("WARNING", "Please, insert numbers only in BMI field!")

    smoking_status_info = smoking_status_entry.get()
    #stroke_info = stroke_entry.get()

   #lista do liczenia prawdopodobieństwa
    lista_slownik = []

    results["gender"] = gendername_info
    lista_slownik.append(gendername_info)
    results["age"] = age_info
    lista_slownik.append(age_info)
    results["hypertension"] = hypertension_info
    lista_slownik.append(hypertension_info)
    results["heart disease"] = heart_disease_info
    lista_slownik.append(heart_disease_info)
    results["ever married"] = ever_married_info
    lista_slownik.append(ever_married_info)
    results["work type"] = work_type_info
    lista_slownik.append(work_type_info)
    results["residence type"] = residence_type_info
    lista_slownik.append(residence_type_info)
    results["avg glucose level"] = avg_glucose_level_info
    lista_slownik.append(avg_glucose_level_info)
    results["bmi"] = bmi_info
    lista_slownik.append(bmi_info)
    results["smoking status"] = smoking_status_info
    lista_slownik.append(smoking_status_info)
    #results["stroke"] = stroke_info
    #lista_slownik.append(stroke_info)

    mandatory_fields()


    print(lista_slownik) #lista wszystkich wprowadzonych danych
    # zmiana gender 1,2 = Male, Female
    if lista_slownik[0] == "1": 
        lista_slownik[0] = "Male"
    else:
        lista_slownik[0] = "Female"
    # 'dopasowanie wieku do grupy wiekowej'
    if lista_slownik[1] < 20:   
        lista_slownik[1] = "nastolatki"
    elif lista_slownik[1] < 30:
        lista_slownik[1] = "20latki"
    elif lista_slownik[1] < 40:
        lista_slownik[1] = "30latki"
    elif lista_slownik[1] < 50:
        lista_slownik[1] = "40latki"
    elif lista_slownik[1] < 60:
        lista_slownik[1] = "50latki"
    elif lista_slownik[1] < 70:
        lista_slownik[1] = "60latki"
    else:
        lista_slownik[1] = "70+"

    #tworzenie listy filtrów po których będzie liczone prawdopodobieństwo
    global lista_p
    lista_p = [lista_slownik[1], lista_slownik[2], lista_slownik[3]] 
    print(lista_p)
    
    #funkcja do obliczania BMI
def show_bmi():
    bmi_entry.delete(0, END)
    x = round((float(weight_entry.get())/float(height_entry.get())**2),2)
    bmi_entry.insert(0,x)
   
    #widok okna do pobrania danych BMI
def bmi_data():
    global weight_entry
    global height_entry
    bmi_screen = Toplevel()
    bmi_screen.title("BMI more info")
    bmi_screen.iconbitmap("C:/Users/Blazej/anaconda3/Python/jdszr6-materialy-python-master/stroke/WDA.ico")
    bmi_screen.minsize(400, 280)
    bmi_screen.config(bg="light blue")
    Text_bmi = LabelFrame(bmi_screen, text="Body mass index", width=220, font = "Helvetica 10 bold", bg = "light blue", borderwidth = 3)
    Text_bmi.place(x=35, y=8)
    bmi_description = Label(Text_bmi, bg = "light blue" ,text="""
    Body Mass Index (BMI) is a person's weight in kilograms 
    divided by the square of height in meters. A high BMI can 
    indicate high body fatness. BMI screens for weight 
    categories that may lead to health problems, but it does 
    not diagnose the body fatness or health of an individual.
    """).pack()
    height_value = Label(bmi_screen, text="Height [m]", font="Helvetica 8 bold", bg = "light blue")
    height_value.place(x=50, y=160)
    height_entry = Entry(bmi_screen, textvariable=height, bd=3)
    height_entry.place(x=130, y =160, width = "100")
    weight_value = Label(bmi_screen, text="Weight [kg]", font="Helvetica 8 bold", bg = "light blue")
    weight_value.place(x=50, y=190)
    weight_entry = Entry(bmi_screen, textvariable=weight, bd=3)
    weight_entry.place(x=130, y =190, width = "100")
    btn_calculate = Button(bmi_screen, text="Submit", width="13", command= show_bmi)
    btn_calculate.pack()
    btn_calculate.place(x= 130, y = 230)
    

def screen_2():

    global hypertension_entry
    global heart_disease_entry
    global ever_married_entry
    global work_type_entry
    global residence_type_entry
    global smoking_status_entry
    global bmi_entry
    #global stroke_entry


    top = Toplevel()
    top.title("World Data Analysis")
    top.geometry("500x500")
    top.resizable(False,False)
    top.config(bg = "light blue")
    top.iconbitmap("C:/Users/Blazej/anaconda3/Python/jdszr6-materialy-python-master/stroke/WDA.ico")
    heading_2 = Label(top, text="HEALTH FORM", bg="navy", fg="light blue", width="500", height="3", font="Helvetica 18 bold")
    heading_2.pack()

    gender_name = Label(top, text="Gender", bg = "light blue", font="Helvetica 8 bold")
    gender_name.place(x=70, y=100)
    gendername_entry_1 = Radiobutton(top, text="Male", bg = "light blue", variable=gender, value="Male")
    gendername_entry_1.place(x=140, y=100, width="150")
    gendername_entry_2 = Radiobutton(top, text="Female", bg = "light blue", variable=gender, value="Female")
    gendername_entry_2.place(x=240, y=100, width="150")

    age_number = Label(top, text="Age", font="Helvetica 8 bold", bg = "light blue")
    age_number.place(x=70, y=130)
    age_entry = Entry(top, textvariable=age, bd=3)
    age_entry.delete(0, END)
    age_entry.place(x=190, y=130, width="145")
    hypertension_value = Label(top, text="Hypertension", font="Helvetica 8 bold", bg = "light blue")
    hypertension_value.place(x=70, y=160)
    hypertension_entry = Combobox(top, values=yes_no)
    hypertension_entry.place(x=190, y=160)
    heart_disease_value = Label(top, text="Heart disease", font="Helvetica 8 bold", bg = "light blue")
    heart_disease_value.place(x=70, y=190)
    heart_disease_entry = Combobox(top, values=yes_no)
    heart_disease_entry.place(x=190, y=190)
    ever_married_value = Label(top, text="Ever married", font="Helvetica 8 bold", bg = "light blue")
    ever_married_value.place(x=70, y=220)
    ever_married_entry = Combobox(top, values=yes_no)
    ever_married_entry.place(x=190, y=220)
    work_type_value = Label(top, text="Work type", font="Helvetica 8 bold", bg = "light blue")
    work_type_value.place(x=70, y=250)
    work_type_entry = Combobox(top, values=work_type_options)
    work_type_entry.place(x=190, y=250)  
    residence_type_value = Label(top, text="Residence type", font="Helvetica 8 bold", bg = "light blue")
    residence_type_value.place(x=70, y=280)
    residence_type_entry = Combobox(top, values=residence_type_options)
    residence_type_entry.place(x=190, y=280)
    avg_glucose_level_value = Label(top, text="Avg glucose [mg/dL]", font="Helvetica 8 bold", bg = "light blue")
    avg_glucose_level_value.place(x=70, y=310)
    avg_glucose_level_entry = Entry(top, textvariable=avg_glucose_level, bd=3)
    avg_glucose_level_entry.delete(0, END)
    avg_glucose_level_entry.place(x=190, y=310, width="145")
    bmi_value = Label(top, text="BMI indicator", font="Helvetica 8 bold", bg = "light blue")
    bmi_value.place(x=70, y=340)
    bmi_entry = Entry(top, textvariable=bmi, bd=3)
    bmi_entry.delete(0, END)
    bmi_entry.place(x=190, y=340, width="145")
    btn_bmi = Button(top, text = "?", width = 2, command = bmi_data )
    btn_bmi.place(x=340, y=340)
    smoking_status_value = Label(top, text="Smoking status", font="Helvetiva 8 bold", bg = "light blue")
    smoking_status_value.place(x=70, y=370)
    smoking_status_entry = Combobox(top, values=smoking_status_options)
    smoking_status_entry.place(x=190, y=370)
    # stroke_value = Label(top, text="Stroke", font="Helvetica 8 bold")
    # stroke_value.place(x=80, y=370)
    # stroke_entry = Combobox(top, values=yes_no)
    # stroke_entry.place(x=190, y=370, width="144")
        
    btn = Button(top, text="Submit", command= lambda: get_data())
    btn.place(x=20, y=425, width="110")   

    btn_2 = Button(top, text="Go to report", command=report_results)
    btn_2.place(x=20, y=460, width="110")

    close_btn = Button(top, text="Close", command=top.destroy)
    close_btn.place(x=370, y=460, width="110")      


def mandatory_fields():

    if results["hypertension"] == "" and results["heart disease"] != "":
        warning_msg = messagebox.showinfo("WARNING", "Hypertension field is mandatory. Please fill it.")
        return warning_msg
    elif results["hypertension"] != "" and results["heart disease"] == "":
        warning_msg = messagebox.showinfo("WARNING", "Heart disease filed is mandatory. Please, fill it")
        return warning_msg
    elif results["hypertension"] == "" and results["heart disease"] == "":
        warning_msg = messagebox.showinfo("WARNING", "Hypertension and Heart disease fields are mandatory. Please, fill them both.")
        return warning_msg

    
# def age_filter():
#     df = pd.read_csv("healthcare-dataset-stroke-data.csv")
    
#     if results["age"] <= 19:
#         df1 = df[(df["age"] <= 19) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >=20 and results["age"] <=29:
#         df1 = df[(df["age"] > 19) & (df["age"] <= 29) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >=30 and results["age"] <=39:
#         df1 = df[(df["age"] >= 30) & (df["age"] <= 39) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])]  
#         return df1
#     if results["age"] >=40 and results["age"] <=49:
#         df1 = df[(df["age"] >= 40) & (df["age"] <= 49) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >=50 and results["age"] <=59:
#         df1 = df[(df["age"] >= 50) & (df["age"] <= 59) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >=60 and results["age"] <=69:
#         df1 = df[(df["age"] >= 60) & (df["age"] <= 69) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >=70 and results["age"] <=79:
#         df1 = df[(df["age"] >= 70) & (df["age"] <= 79) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1
#     if results["age"] >= 80 and results["age"] <=89:
#         df1 = df[(df["age"] >= 80) & (df["age"] <= 89) & (df["gender"] == results["gender"]) & (df["hypertension"] == results["hypertension"]) & (df["heart_disease"] == results["heart disease"])] 
#         return df1

def report_description():
    
    text_frame = LabelFrame(top2, text="Results description", font = "Helvetica 10 bold", bg = "light blue", borderwidth = 3)
    text_frame.pack()
    text_frame.place(x=25, y=101)
    # len(age_filter())
    # sum(age_filter()["stroke"])

    # try:
    #     final_score = (sum(age_filter()["stroke"]) / len(age_filter()) * 100)

    # except ZeroDivisionError:
    #     wynik = Label(text_frame, text=f"""
    #     Twoje prawopodobieństwo wystąpienia udaru wynosi 0.0 %.
    #     Gratulujemy i prosimy wykonać badanie ponownie za rok.
    #     """)
    #     wynik.pack()

    if p == 0:
        wynik = Label(text_frame, bg = "light blue", text=f"""
        Twoje prawopodobieństwo wystąpienia udaru wynosi {p:.2%}.
        Gratulujemy i prosimy wykonać badanie ponownie za rok.
        """)
        wynik.pack()
    elif p > 0 and p <= 20:
        wynik = Label(text_frame, bg = "light blue", text=f"""
        Twoje prawopodobieństwo wystąpienia udaru wynosi {p:.2%}.
        Ryzyko nie jest wielkie jednak musisz skonultować się z lekarzem
        w celu ustalenia jak możesz chronić się przed udarem.
        
        Aby zarejestrować się do lekarza specjalisty prosimy dzwonić pod numer:

        +48 775 667 123
        """)
        wynik.pack()
    elif p > 20 and p < 50:
        wynik = Label(text_frame, bg = "light blue", text=f"""
        Twoje prawopodobieństwo wystąpienia udaru wynosi {p:.2%}.
        Prosimy umów się na wizytę do lekarza specjalisty, aby wykonać 
        dodatkowe badania. Lekarz udzieli również niezbędnych porad
        co zrobić, aby znacznie zmniejszyć ryzyko udaru.
        
        Aby zarejestrować się do lekarza specjalisty prosimy dzwonić pod numer:

        +48 775 667 123
        """)
        wynik.pack()
    else:
        wynik = Label(text_frame, bg = "light blue", text=f"""
        Twoje prawopodobieństwo wystąpienia udaru wynosi {p:.2%}.
        Niezbędna jest natychmiastowa wizyta u lekarza specjalisty.
        Możesz również wymagać hospitalizacji.
        Proszę zgłoś się do lekarza, który oceni Twój stan zdrowia.
        
        Aby zarejestrować się do lekarza specjalisty prosimy dzwonić pod numer:

        +48 775 667 123
        """)
        wynik.pack()

    # Wykres BMI
    x1=["Body mass index"]
    y1=[50]

    x2=["Body mass index"]
    y2=[30]

    x3=["Body mass index"]
    y3=[25]

    x4=["Body mass index"]
    y4=[18,5]
    
    y5= float(bmi_entry.get())

    width = 1
    fig, ax=plt.subplots(figsize=(10,8))
    ax=plt.bar(x1,y1,label='obesity', color = "0.2")
    ax=plt.bar(x2,y2,label='overweight', color = "0.31")
    ax=plt.bar(x3,y3,label='normal', color = "green")
    ax=plt.bar(x4,y4,label='underweight', color = "0.5")
    plt.axhline(y5,label = "your BMI", color = "red")

    plt.title("BMI")
    plt.ylabel("Values")

    plt.legend(loc='upper right')
    plt.show()


def report_results():
    #liczenie prawdopodobieństwa
    b1 = len(file[(file["heart_disease"]== lista_p[2]) & (file["age_group"] == lista_p[0]) & (file["hypertension"] == lista_p[1])]) 
    a1= len(file[(file["heart_disease"]== lista_p[2]) & (file["age_group"] == lista_p[0]) & (file["hypertension"] == lista_p[1]) & (file["stroke"]== 1)])
    global p
    p = a1/b1


    global top2
    
    top2 = Toplevel()
    top2.title("World Data Analysis")
    top2.geometry("500x500")
    top2.resizable(False,False)
    top2.config(bg = "light blue")
    top2.iconbitmap("C:/Users/Blazej/anaconda3/Python/jdszr6-materialy-python-master/stroke/WDA.ico")
    heading_3 = Label(top2, text="HEALTH REPORT", bg="navy", fg="light blue", width="500", height="3", font="Helvetica 18 bold")
    heading_3.pack()

    run_report = Button(top2, text="Run report", width="16", command=report_description)
    run_report.place(x=20, y=460)

    execute_btn = Button(top2, text="Close", width="16", command=top2.destroy)
    execute_btn.place(x=360, y=460)


execute_btn = Button(window, text="Open form", width="16", height="3", font="Helvetica 11 bold", command=screen_2)
execute_btn.place(x=170, y=250)

close_btn = Button(window, text="Close", command=window.destroy)
close_btn.place(x=360, y=400, width="122")

window.mainloop()

In [ ]:
pip install pyqt5